In [1]:
import numpy as np
import math
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

The aim is to :<br>
- check that we have training data for all (id, meter) that are in the test set.
- have a look at what we have to predict.

In [6]:
train_df = pd.read_csv('../data/raw/csvs/train.csv', parse_dates=['timestamp'])

In [7]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [8]:
test_df = pd.read_csv('../data/raw/csvs/test.csv', parse_dates=['timestamp'])

In [49]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
row_id         int64
building_id    int64
meter          int64
timestamp      datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.2 GB


In [9]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


In [11]:
test_df.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01
1,1,1,0,2017-01-01
2,2,2,0,2017-01-01
3,3,3,0,2017-01-01
4,4,4,0,2017-01-01


In [12]:
# 2 years to predict
print('min date : {}'.format(test_df['timestamp'].min()))
print('max date : {}'.format(test_df['timestamp'].max()))

min date : 2017-01-01 00:00:00
max date : 2018-12-31 23:00:00


In [15]:
train_building_ids = train_df['building_id']
print('n train buildings : {}'.format(train_building_ids.nunique()))

test_building_ids = test_df['building_id']
print('n test buildings : {}'.format(test_building_ids.nunique()))

n train buildings : 1449
n test buildings : 1449


In [22]:
# Building lists are equals

train_building_set = set(train_building_ids.unique())
test_building_set = set(test_building_ids.unique())
train_building_set == test_building_set

True

<b>Count n training values for each (building_id, meter)</b>

In [45]:
train_df_grouped = train_df.groupby(['building_id', 'meter']).count()
train_df_grouped.drop('timestamp', axis=1, inplace=True)
train_df_grouped.rename({'meter_reading' : 'n_meter_readings'}, axis=1, inplace=True)
train_df_grouped.sort_values(by='n_meter_readings', axis=0, ascending=False, inplace=True)

In [46]:
train_df_grouped.head()

,,n_meter_readings
building_id,meter,
0,0,8784
685,0,8784
672,0,8784
673,0,8784
674,0,8784


In [48]:
# We have a few problematic meters with a small number of values.
# We will probably need a more naive predictor for those.
train_df_grouped.tail(15)

,,n_meter_readings
building_id,meter,
647,0,3328
568,0,3298
621,0,3062
1024,0,2981
1412,2,2844
591,0,2747
1017,0,2696
783,2,2657
420,0,2327
